# Multi-Dataset Consolidation with XNSampleProcessor

This notebook demonstrates how to use `XNSampleProcessor` to consolidate and clean multiple XN_SAMPLE.csv files from different sources or decryptions.

## Use Cases

1. **Multiple decryptions**: Combine data from multiple `.116` file decryptions
2. **Multi-site studies**: Consolidate data from different hospitals/analysers
3. **Longitudinal studies**: Merge data collected at different time points
4. **Batch processing**: Process multiple datasets with consistent parameters

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import sys

# Add parent directory to path if needed
sys.path.insert(0, str(Path.cwd().parent.parent))

from sysmexcbctools.data import XNSampleProcessor

# Plotting imports and styling (used in later cells)
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# PDF-compatible fonts
matplotlib.rcParams["pdf.fonttype"] = 42
matplotlib.rcParams["ps.fonttype"] = 42

# Scientific plot style
import scienceplots

plt.style.use(["science", "nature"])

# Colourblind-friendly palette
SEABORN_PALETTE = "colorblind"
seaborn_colors = sns.color_palette(SEABORN_PALETTE)

## Test Data

For this notebook, we'll load a single XN_SAMPLE.csv file and artificially partition it into three parts to simulate multiple decryptions or data sources. This allows us to demonstrate multi-file consolidation functionality without requiring multiple actual source files.

### Loading Data with ConfigLoader

First, we'll load the data using the same approach as in the basic cleaning notebook - using the YAML config with fallback to manual path specification.

In [ ]:
import sys
from pathlib import Path

# Add parent directory to path to import sysmexcbctools
project_root = Path.cwd().parent.parent
sys.path.insert(0, str(project_root))

# Path to your XN_SAMPLE.csv file
# Option 1: Use the config loader (if you have the data_paths.yaml configured)
try:
    from sysmexcbctools.transfer.config.config_loader import ConfigLoader

    config_loader = ConfigLoader(
        config_file=str(
            project_root / "sysmexcbctools/transfer/config/data_paths.yaml"
        ),
        environment="production",
    )

    # Get the raw data directory for INTERVAL dataset 36
    dataset_dir = config_loader.get_dataset_path(category="raw", dataset="interval_36")
    data_path = dataset_dir / "XN_SAMPLE.csv"

    print(f"✓ Loaded path from config: {data_path}")

except (FileNotFoundError, ValueError, KeyError) as e:
    print(f"⚠ Could not load from config: {e}")
    print("  Falling back to manual path specification...")

    # Option 2: Manually specify your data path
    # EDIT THIS PATH to point to your XN_SAMPLE.csv file:
    data_path = Path("/path/to/your/XN_SAMPLE.csv")

    print(f"\n  Please edit this cell and set data_path to your file location.")
    print(f"  Current (placeholder) path: {data_path}")

print(f"\nFile exists: {data_path.exists()}")
if data_path.exists():
    print(f"File size: {data_path.stat().st_size / 1024**2:.1f} MB")

In [ ]:
# Load the full dataset
print("Loading full dataset...")
df_full = pd.read_csv(data_path, encoding="ISO-8859-1", low_memory=False)
print(f"Loaded {len(df_full):,} rows")

# Split into three parts by date (to simulate different decryptions/time periods)
# Sort by date first to make a realistic split
if "Date" in df_full.columns:
    df_full = df_full.sort_values("Date")
    split_method = "chronological (by date)"
else:
    # If no date column, just split sequentially
    split_method = "sequential"

n = len(df_full)
split_1 = n // 3
split_2 = 2 * n // 3

df_part1 = df_full.iloc[:split_1].copy()
df_part2 = df_full.iloc[split_1:split_2].copy()
df_part3 = df_full.iloc[split_2:].copy()

print(f"\nSplit method: {split_method}")
print(f"Part 1: {len(df_part1):,} rows ({len(df_part1)/n*100:.1f}%)")
print(f"Part 2: {len(df_part2):,} rows ({len(df_part2)/n*100:.1f}%)")
print(f"Part 3: {len(df_part3):,} rows ({len(df_part3)/n*100:.1f}%)")

# Save to temporary files
temp_dir = Path("../data/temp")
temp_dir.mkdir(parents=True, exist_ok=True)

file_list = [
    str(temp_dir / "XN_SAMPLE_part1.csv"),
    str(temp_dir / "XN_SAMPLE_part2.csv"),
    str(temp_dir / "XN_SAMPLE_part3.csv"),
]

print(f"\nSaving temporary files to: {temp_dir}")
df_part1.to_csv(file_list[0], index=False)
df_part2.to_csv(file_list[1], index=False)
df_part3.to_csv(file_list[2], index=False)

print("✓ Test data partitions created successfully!")
print(f"\nFile list for processing:")
for f in file_list:
    print(f"  - {f}")

## 1. Consolidating Multiple Files Directly

The simplest approach: provide a list of file paths to `process_files()`.

In [ ]:
# Process multiple CSV files at once
# Using the file_list created in the previous cell

# Create processor
processor = XNSampleProcessor(verbose=1)  # Show progress messages

# Process all files together
df_consolidated = processor.process_files(
    input_files=file_list, dataset_name="consolidated", save_output=False
)

print(f"\nConsolidated dataset shape: {df_consolidated.shape}")
print(f"Unique samples: {df_consolidated['Sample No.'].nunique()}")
if "Date" in df_consolidated.columns:
    print(
        f"Date range: {df_consolidated['Date'].min()} to {df_consolidated['Date'].max()}"
    )

### What happens during consolidation?

1. **Files are concatenated** before processing
2. **Duplicate rows** are removed (identical across all columns)
3. **Multiple measurements** per sample are handled according to your settings
4. **All cleaning steps** are applied to the combined dataset

## 2. Tracking Data Sources

When combining multiple files, you may want to track which samples came from which source.

In [ ]:
# Load files manually to add source tracking
dfs = []
sources = ["Part_1", "Part_2", "Part_3"]

for file_path, source in zip(file_list, sources):
    df = pd.read_csv(file_path)
    df["DataSource"] = source  # Add source identifier
    dfs.append(df)
    print(f"Loaded {len(df)} rows from {source}")

# Concatenate
df_combined = pd.concat(dfs, ignore_index=True)
print(f"\nTotal rows before cleaning: {len(df_combined)}")

# Save temporarily
temp_file = "../data/temp_combined.csv"
df_combined.to_csv(temp_file, index=False)

# Process with source tracking preserved
processor = XNSampleProcessor()
df_clean = processor.process_files(temp_file, save_output=False)

# Check source distribution
print("\nSamples per source after cleaning:")
print(df_clean["DataSource"].value_counts())

## 3. Config-Based Multi-Dataset Processing

For complex projects with many datasets, use a YAML configuration file to define all your data sources and processing parameters.

In [ ]:
# Example config structure
config_example = """
input:
  datasets:
    - name: INTERVAL_baseline
      files:
        - /path/to/INTERVAL/XN_SAMPLE_baseline.csv
    
    - name: INTERVAL_followup
      files:
        - /path/to/INTERVAL/XN_SAMPLE_followup.csv
    
    - name: STRIDES
      files:
        - /path/to/STRIDES/decryption1/XN_SAMPLE.csv
        - /path/to/STRIDES/decryption2/XN_SAMPLE.csv
        - /path/to/STRIDES/decryption3/XN_SAMPLE.csv

processing:
  remove_clotintube: true
  remove_multimeasurementsamples: true
  std_threshold: 1.0
  remove_correlated: false

output:
  directory: ./processed_data
  filename_prefix: XN_SAMPLE_clean
"""

print(config_example)

In [ ]:
# Create a real config file for this example
import yaml

# Use the file_list we created earlier
config = {
    "input": {
        "datasets": [
            {"name": "part1", "files": [file_list[0]]},
            {"name": "part2", "files": [file_list[1]]},
            {"name": "part3", "files": [file_list[2]]},
        ]
    },
    "processing": {
        "remove_clotintube": True,
        "remove_multimeasurementsamples": True,
        "std_threshold": 1.0,
    },
    "output": {"directory": "./output", "filename_prefix": "XN_SAMPLE_processed"},
}

config_path = str(temp_dir / "multi_dataset_config.yaml")
with open(config_path, "w") as f:
    yaml.dump(config, f)

print(f"Created config file: {config_path}")
print("\nConfig contents:")
print(yaml.dump(config, default_flow_style=False))

In [ ]:
# Process using config file
processor = XNSampleProcessor(config_path=config_path)

# Process specific dataset
df_part1 = processor.process("part1")
print(f"\nPart 1 shape: {df_part1.shape}")

df_part2 = processor.process("part2")
print(f"Part 2 shape: {df_part2.shape}")

## 4. Batch Processing Multiple Datasets

Process all datasets defined in your config with consistent parameters.

In [ ]:
# Process all datasets in config
processor = XNSampleProcessor(
    config_path=config_path,
    log_to_file=True,  # Save logs and diagnostics for each dataset
)

results = {}
for dataset_config in processor.config["input"]["datasets"]:
    dataset_name = dataset_config["name"]

    try:
        print(f"\n{'='*50}")
        print(f"Processing: {dataset_name}")
        print(f"{'='*50}")

        df = processor.process(dataset_name)
        results[dataset_name] = df

        print(
            f"✓ Completed: {df.shape[0]} rows, {df['Sample No.'].nunique()} unique samples"
        )

    except Exception as e:
        print(f"✗ Failed: {e}")

# Summary
print(f"\n\n{'='*50}")
print("BATCH PROCESSING SUMMARY")
print(f"{'='*50}")
for name, df in results.items():
    print(f"{name:20s}: {df.shape[0]:6d} rows, {df['Sample No.'].nunique():6d} samples")

## 5. Handling Overlapping Samples

What if the same sample appears in multiple files? The processor handles this automatically.

In [ ]:
# Check for duplicate sample IDs across files
sample_ids_by_file = []

for file_path in file_list:
    df = pd.read_csv(file_path)
    sample_ids = set(df["Sample No."].unique())
    sample_ids_by_file.append(sample_ids)
    print(f"{Path(file_path).name}: {len(sample_ids)} unique samples")

# Find overlaps
all_samples = set.union(*sample_ids_by_file)
overlapping_samples = set()
for i in range(len(sample_ids_by_file)):
    for j in range(i + 1, len(sample_ids_by_file)):
        overlap = sample_ids_by_file[i] & sample_ids_by_file[j]
        overlapping_samples.update(overlap)

print(f"\nTotal unique samples across all files: {len(all_samples)}")
print(f"Samples appearing in multiple files: {len(overlapping_samples)}")

# Process and check
processor = XNSampleProcessor()
df_clean = processor.process_files(file_list, save_output=False)

print(f"\nAfter processing:")
print(f"  Total rows: {len(df_clean)}")
print(f"  Unique samples: {df_clean['Sample No.'].nunique()}")
print(
    f"\nDuplicate handling: {'remove_duplicate_rows()' if len(df_clean) < sum(len(pd.read_csv(f)) for f in file_list) else 'no duplicates found'}"
)

### How duplicate samples are handled:

1. **Exact duplicates** (identical across all columns) → removed by `remove_duplicate_rows()`
2. **Multiple measurements** (same Sample No., different values) → handled by `handle_multiple_measurements()`
   - If measurements agree (within std_threshold) → keep earliest
   - If measurements disagree → flagged for manual review (saved to diagnostic file if `log_to_file=True`)

## 6. Custom Parameters for Different Datasets

Sometimes you need different processing parameters for different datasets.

In [ ]:
# Process dataset with strict parameters
processor_strict = XNSampleProcessor(
    remove_clotintube=True,
    remove_multimeasurementsamples=True,
    std_threshold=0.5,  # More strict - fewer measurements will be considered "matching"
    verbose=1,
)

df_strict = processor_strict.process_files(
    file_list[0],  # Using first partition from file_list
    dataset_name="part1_strict",
    save_output=False,
)

# Process same dataset with lenient parameters
processor_lenient = XNSampleProcessor(
    remove_clotintube=False,  # Keep clotted samples
    remove_multimeasurementsamples=True,
    std_threshold=2.0,  # More lenient - more measurements will be considered "matching"
    verbose=1,
)

df_lenient = processor_lenient.process_files(
    file_list[0],  # Using first partition from file_list
    dataset_name="part1_lenient",
    save_output=False,
)

print(f"\nStrict processing: {len(df_strict)} rows")
print(f"Lenient processing: {len(df_lenient)} rows")
print(
    f"Difference: {len(df_lenient) - len(df_strict)} rows ({(len(df_lenient) - len(df_strict))/len(df_strict)*100:.1f}%)"
)

## 7. Comparing Datasets After Processing

Once you've processed multiple datasets, you may want to compare them.

In [ ]:
# Compare FBC parameter distributions
# Use datasets from section 4, or create fresh if needed
if "results" not in locals() or not results:
    print("Loading datasets for comparison...")
    processor = XNSampleProcessor()
    results = {}
    for i, part_file in enumerate(file_list[:2], 1):  # Use first two partitions
        part_name = f"XN_SAMPLE_part{i}"
        results[part_name] = processor.process_files(part_file, save_output=False)

fig, axes = plt.subplots(2, 3, figsize=(6.6, 4.5))
axes = axes.flatten()

fbc_params = [
    r"WBC(10$^3$/uL)",
    r"RBC(10$^6$/uL)",
    r"HGB(g/dL)",
    r"PLT(10$^3$/uL)",
    r"MCV(fL)",
    r"MCH(pg)",
]

# Map display names to actual column names in the dataframe
param_column_map = {
    r"WBC(10$^3$/uL)": "WBC(10^3/uL)",
    r"RBC(10$^6$/uL)": "RBC(10^6/uL)",
    r"HGB(g/dL)": "HGB(g/dL)",
    r"PLT(10$^3$/uL)": "PLT(10^3/uL)",
    r"MCV(fL)": "MCV(fL)",
    r"MCH(pg)": "MCH(pg)",
}

for i, param_display in enumerate(fbc_params):
    ax = axes[i]
    param_column = param_column_map[param_display]

    # Check if parameter exists in any dataset
    plotted_any = False
    for name, df in results.items():
        if param_column in df.columns:
            data = df[param_column].dropna()
            if len(data) > 0:
                ax.hist(
                    data,
                    alpha=0.5,
                    label=name,
                    bins=50,
                    edgecolor="black",
                    linewidth=0.5,
                )
                plotted_any = True

    if plotted_any:
        ax.set_xlabel(param_display)
        ax.set_ylabel("Frequency")
        ax.legend()
        ax.set_title(f"{param_display} Distribution")
        ax.grid(True, alpha=0.3)
    else:
        ax.text(
            0.5,
            0.5,
            f"{param_display}\nNo data available",
            ha="center",
            va="center",
            transform=ax.transAxes,
        )
        ax.set_xticks([])
        ax.set_yticks([])

plt.tight_layout()
plt.show()

## 8. Merging Processed Datasets

After processing datasets separately, you might want to combine them for joint analysis.

In [ ]:
# Process datasets separately first
processor = XNSampleProcessor()

df1 = processor.process_files(
    file_list[0], dataset_name="part1", save_output=False  # First partition
)
df1["data_source"] = "Part 1"

df2 = processor.process_files(
    file_list[1], dataset_name="part2", save_output=False  # Second partition
)
df2["data_source"] = "Part 2"

# Merge
df_merged = pd.concat([df1, df2], ignore_index=True)

print(f"\nMerged dataset:")
print(f"  Total rows: {len(df_merged)}")
print(f"  Unique samples: {df_merged['Sample No.'].nunique()}")
print(f"\nSamples by source:")
print(df_merged["data_source"].value_counts())

## 9. Saving Processed Datasets

Options for saving your processed data.

In [ ]:
# Option 1: Save automatically during processing
processor = XNSampleProcessor(
    output_dir="./processed_data",
    output_prefix="study_clean",
    log_to_file=True,  # Also save logs and diagnostics
)

df = processor.process_files(
    file_list, dataset_name="my_study", save_output=True  # Automatically saves
)

print(f"\nSaved to: {processor.output_dir}/study_clean_my_study_YYYYMMDD_HHMMSS.csv")

In [ ]:
# Option 2: Save manually after processing
processor = XNSampleProcessor()
df = processor.process_files(file_list, save_output=False)

# Save with custom name
output_path = "./my_custom_output.csv"
df.to_csv(output_path, index=False)
print(f"Saved to: {output_path}")

# Or save in other formats
df.to_parquet("./my_output.parquet", index=False)
df.to_pickle("./my_output.pkl")

## Summary

### Key Points for Multi-Dataset Processing:

1. **Direct file lists**: Simple and flexible for quick processing
2. **Config files**: Best for complex projects with many datasets
3. **Source tracking**: Add identifier columns before processing
4. **Duplicate handling**: Automatic removal and flagging
5. **Batch processing**: Process multiple datasets consistently
6. **Custom parameters**: Tailor processing to each dataset's needs
7. **Merging**: Combine processed datasets for joint analysis

### Best Practices:

- Always check for overlapping samples across datasets
- Use config files for reproducibility
- Enable `log_to_file=True` for important processing runs
- Review diagnostic files for samples with discrepant measurements
- Save intermediate results to avoid reprocessing
- Document data sources and processing parameters

In [ ]:
# Cleanup temporary files (uncomment to execute)
# import shutil
# if temp_dir.exists():
#     shutil.rmtree(temp_dir)
#     print(f"✓ Removed temporary directory: {temp_dir}")
# else:
#     print("No temporary files to clean up")

print("(Uncomment the code above to remove temporary files)")